In [1]:
#%pip install ipykernel optuna lightgbm plotly scikit-learn mlflow optuna_integration

In [ ]:
%pip install sacrebleu

In [2]:

import numpy as np
import optuna
from pprint import pprint 
import matplotlib.pyplot as plt
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
import plotly
import pandas as pd
from sklearn.model_selection import train_test_split
from sacrebleu import sentence_bleu

d:\.VScode\SuperAI4\Image_captioning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class Blue_Score():
    def __init__(self,prediction,reference):
        self.pred = prediction
        self.reff = reference
        self.Accumulate_Bleuscore = []
    def run(self,pattern = 0):
        self.pattern = pattern
        try:
            for pred,refs in zip(self.pred,self.reff):
                score = sentence_bleu(pred, refs, tokenize="flores101")
                if pattern == 0:
                    print(score)
                elif pattern == 1:
                    print(f"BLEU score : '{pred}': {score.score:.2f}")
                elif pattern == 2:
                    print(f"BLEU score -> Ref : {refs}\tPred : {pred}': {score.score:.2f}")
                elif pattern == 3:
                    pass
                self.Accumulate_Bleuscore.append(score.score)
        except:
            print("Error some where")
    def runavg(self):
        self.run(3)
        return sum(self.Accumulate_Bleuscore)/len(self.Accumulate_Bleuscore)

In [ ]:
import torch
n_sample = 2000
device = "cuda" if torch.cuda.is_available() else "cpu"
def objective(trial):
    sample = pd.sample(n = n_sample)
    
    pixel_values_list = []
    label_list = []
    for path, label in sample:
        im = Image.open(path).convert("RGB")
        
    pixel_values = processor(im, return_tensors="pt").to(device).pixel_values
    pixel_values_list.append(pixel_values)
    label_list.append(label)
        
    pixel_values = torch.stack(pixel_values_list)
    param = {
        "max_length": trial.suggest_int("max_length", 50, 70, log=True),
        "min_length": trial.suggest_int("min_length", 20, 40, log=True),
        "do_sample":trial.suggest_categorical("do_sample", log=True), 
        "num_beams": trial.suggest_int("num_beams", 3, 5, log=True),
        "top_k": trial.suggest_int("top_k", 10, 30, log=True),
        "temperature": trial.suggest_float("temperature", 1.0, 3.0, log=True),
        "length_penalty": trial.suggest_float("length_penalty", 1.0, 3.0, log=True),
        "num_beam_groups": trial.suggest_int("num_beam_groups", 1.0, 3.0, log=True),
        "diversity_penalty": trial.suggest_float("diversity_penalty", 1.0, 3.0, log=True),
        "repetition_penalty": trial.suggest_float("repetition_penalty", 1.0, 3.0, log=True),
        "length_penalty": trial.suggest_float("length_penalty", 1.0, 3.0, log=True),
        "no_repeat_ngram_size": trial.suggest_int("no_repeat_ngram_size", 2, 5, log=True)
    }
    model.eval()
    
    outputs = model.generate(
        pixel_values=pixel_values,
        num_beam_groups = 5,
        **param
    ).cpu().numpy().tolist()
    
    bleu = Blue_Score(outputs, label_list).runavg()
    return bleu

In [4]:
class verbosityCallback:
    def __init__(self, model):
        self.model = model
        
    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial) -> None:
        if study.best_trial == trial:
            ## TODO: save model
            pass

In [7]:


# Add stream handler of stdout to show the messages
optuna.logging.set_verbosity(optuna.logging.WARNING)
n_trials = 50
study_stop_cb = verbosityCallback(model)

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.SuccessiveHalvingPruner(reduction_factor=4),
    sampler=optuna.samplers.RandomSampler(),
)
study.optimize(objective, n_trials=n_trials, callbacks=[study_stop_cb], n_jobs=-1, show_progress_bar=True)
plot_study(study)

Best trial: 6. Best value: 0.982456: 100%|██████████| 100/100 [00:03<00:00, 32.46it/s]


Best trial: 6 with value: 0.9824561403508771
("Best params: {'lambda_l1': 2.3883141049693244e-08, 'lambda_l2': "
 "2.9758116097509773e-07, 'num_leaves': 208, 'feature_fraction': "
 "0.8957768077257513, 'bagging_fraction': 0.9180711062834542, 'bagging_freq': "
 "6, 'min_child_samples': 97}")


In [9]:
# Add stream handler of stdout to show the messages
optuna.logging.set_verbosity(optuna.logging.WARNING)
return_list = []
n_trials = 20
study_stop_cb = verbosityCallback(
    model=model
)

study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=n_trials, callbacks=[study_stop_cb], n_jobs=-1, show_progress_bar=True)
plot_study(study)

Best trial: 55. Best value: 0.989474: 100%|██████████| 100/100 [00:08<00:00, 12.02it/s]


Best trial: 55 with value: 0.9894736842105263
("Best params: {'lambda_l1': 1.626568337537173e-07, 'lambda_l2': "
 "2.0447155774425385e-05, 'num_leaves': 58, 'feature_fraction': "
 "0.5941326115326094, 'bagging_fraction': 0.5189313759506909, 'bagging_freq': "
 "5, 'min_child_samples': 28}")


In [10]:
# from sklearn import metrics
# def detailed_objective(trial):
#     # Use same code objective to reproduce the best model
#     X, y = make_classification(n_features=10, random_state=1)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#     C = trial.suggest_float("C", 1e-7, 10.0, log=True)

#     clf = LogisticRegression(C=C)
#     clf.fit(X_train, y_train)

#     # calculate more evaluation metrics
#     pred = clf.predict(X_test)

#     acc = metrics.accuracy_score(pred, y_test)
#     recall = metrics.recall_score(pred, y_test)
#     precision = metrics.precision_score(pred, y_test)
#     f1 = metrics.f1_score(pred, y_test)

#     return acc, f1, recall, precision

# detailed_objective(study.best_trial)  # calculate acc, f1, recall, and precision

# MLflow integration

In [ ]:
import mlflow
uri = "http://127.0.0.1:8080"
mlflow.set_tracking_uri(uri=uri)

In [ ]:
from optuna_integration.mlflow import MLflowCallback

mlflc = MLflowCallback(
    tracking_uri=uri,
    metric_name="Accuracy",
)

study = optuna.create_study(study_name="LightBGM-1")
study.optimize(objective, n_trials=30, callbacks=[mlflc])

C:\Users\Toon\AppData\Local\Temp\ipykernel_1968\832758089.py:3: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
[I 2024-04-22 10:00:22,579] A new study created in memory with name: LightBGM-1
[I 2024-04-22 10:00:22,621] Trial 0 finished with value: 0.965034965034965 and parameters: {'lambda_l1': 0.04271602670515351, 'lambda_l2': 0.029575573100331147, 'num_leaves': 25, 'feature_fraction': 0.48486206576405266, 'bagging_fraction': 0.5338726276100296, 'bagging_freq': 5, 'min_child_samples': 42}. Best is trial 0 with value: 0.965034965034965.
2024/04/22 10:00:22 INFO mlflow.tracking.fluent: Experiment with name 'LightBGM-1' does not exist. Creating a new experiment.
[I 2024-04-22 10:00:22,856] Trial 1 finished with value: 0.951048951048951 and parameters: {'lambda_l1': 5.570496620123377e-07, 'lambda_l2': 0.0009705079548096559, 'num_leaves': 11, 'feature_fraction': 0.510632474882694, 'bagging_fract